# PrimeKG数据探索

本notebook用于探索PrimeKG数据集，特别是药物-疾病关系。

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from data_loader import PrimeKGDataLoader

# 设置样式
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

%matplotlib inline

## 1. 加载数据

In [ ]:
# 初始化数据加载器
data_loader = PrimeKGDataLoader("../data")

# 加载原始数据
df = data_loader.load_raw_data()

print(f"数据形状: {df.shape}")
print(f"列名: {df.columns.tolist()}")
df.head()

## 2. 数据概览

In [ ]:
# 基本统计信息
print("=== 基本统计信息 ===")
print(f"总边数: {len(df):,}")
print(f"唯一节点数: {len(set(df['x_id']) | set(df['y_id'])):,}")
print(f"节点类型数: {len(set(df['x_type']) | set(df['y_type']))}")
print(f"关系类型数: {df['display_relation'].nunique()}")

print("\n=== 节点类型分布 ===")
node_types = list(df['x_type']) + list(df['y_type'])
node_type_counts = pd.Series(node_types).value_counts()
print(node_type_counts)

print("\n=== 关系类型分布 (Top 20) ===")
relation_counts = df['display_relation'].value_counts().head(20)
print(relation_counts)

## 3. 药物-疾病关系分析

In [ ]:
# 筛选药物-疾病关系
drug_disease_df = data_loader.filter_drug_disease_relations(df)

print(f"药物-疾病关系数量: {len(drug_disease_df):,}")
print(f"占总关系的比例: {len(drug_disease_df)/len(df)*100:.2f}%")

# 药物-疾病关系类型分布
drug_disease_relations = drug_disease_df['display_relation'].value_counts()
print("\n药物-疾病关系类型:")
print(drug_disease_relations)

In [ ]:
# 可视化药物-疾病关系类型分布
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 饼图
ax1.pie(drug_disease_relations.values, labels=drug_disease_relations.index, autopct='%1.1f%%')
ax1.set_title('药物-疾病关系类型分布')

# 柱状图
drug_disease_relations.plot(kind='bar', ax=ax2)
ax2.set_title('药物-疾病关系类型计数')
ax2.set_xlabel('关系类型')
ax2.set_ylabel('数量')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. 节点度分布分析

In [ ]:
# 计算节点度
def calculate_node_degrees(df):
    """计算节点度分布"""
    # 统计每个节点的出度和入度
    out_degrees = df.groupby(['x_id', 'x_type']).size().reset_index(name='out_degree')
    in_degrees = df.groupby(['y_id', 'y_type']).size().reset_index(name='in_degree')
    
    # 重命名列以便合并
    out_degrees.columns = ['node_id', 'node_type', 'out_degree']
    in_degrees.columns = ['node_id', 'node_type', 'in_degree']
    
    # 合并出度和入度
    degrees = pd.merge(out_degrees, in_degrees, on=['node_id', 'node_type'], how='outer')
    degrees = degrees.fillna(0)
    degrees['total_degree'] = degrees['out_degree'] + degrees['in_degree']
    
    return degrees

# 计算药物-疾病网络的节点度
drug_disease_degrees = calculate_node_degrees(drug_disease_df)

print("节点度统计:")
print(drug_disease_degrees['total_degree'].describe())

# 按节点类型分组统计
print("\n按节点类型的度分布:")
degree_by_type = drug_disease_degrees.groupby('node_type')['total_degree'].agg(['count', 'mean', 'std', 'max'])
print(degree_by_type)

In [ ]:
# 可视化度分布
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 总体度分布
axes[0, 0].hist(drug_disease_degrees['total_degree'], bins=50, alpha=0.7)
axes[0, 0].set_title('总体度分布')
axes[0, 0].set_xlabel('度')
axes[0, 0].set_ylabel('频次')
axes[0, 0].set_yscale('log')

# 对数度分布
axes[0, 1].hist(np.log10(drug_disease_degrees['total_degree'] + 1), bins=50, alpha=0.7)
axes[0, 1].set_title('对数度分布')
axes[0, 1].set_xlabel('log10(度 + 1)')
axes[0, 1].set_ylabel('频次')

# 按节点类型的度分布
for i, node_type in enumerate(['drug', 'disease']):
    if node_type in drug_disease_degrees['node_type'].values:
        subset = drug_disease_degrees[drug_disease_degrees['node_type'] == node_type]
        axes[1, i].hist(subset['total_degree'], bins=30, alpha=0.7, label=node_type)
        axes[1, i].set_title(f'{node_type.capitalize()} 度分布')
        axes[1, i].set_xlabel('度')
        axes[1, i].set_ylabel('频次')
        axes[1, i].legend()

plt.tight_layout()
plt.show()

## 5. 高度节点分析

In [ ]:
# 找出度最高的节点
top_nodes = drug_disease_degrees.nlargest(20, 'total_degree')

print("度最高的20个节点:")
print(top_nodes[['node_id', 'node_type', 'total_degree']])

# 分别查看药物和疾病的高度节点
print("\n度最高的10个药物:")
top_drugs = drug_disease_degrees[drug_disease_degrees['node_type'] == 'drug'].nlargest(10, 'total_degree')
print(top_drugs[['node_id', 'total_degree']])

print("\n度最高的10个疾病:")
top_diseases = drug_disease_degrees[drug_disease_degrees['node_type'] == 'disease'].nlargest(10, 'total_degree')
print(top_diseases[['node_id', 'total_degree']])

## 6. 关系网络可视化

In [ ]:
# 创建交互式可视化
import plotly.graph_objects as go
import networkx as nx

# 创建一个小的子图进行可视化（选择度最高的节点）
top_50_nodes = set(drug_disease_degrees.nlargest(50, 'total_degree')['node_id'])

# 筛选包含这些节点的边
subset_df = drug_disease_df[
    (drug_disease_df['x_id'].isin(top_50_nodes)) | 
    (drug_disease_df['y_id'].isin(top_50_nodes))
]

print(f"子图包含 {len(subset_df)} 条边")

# 创建NetworkX图
G = nx.Graph()

for _, row in subset_df.iterrows():
    G.add_edge(
        row['x_id'], row['y_id'],
        relation=row['display_relation'],
        x_type=row['x_type'],
        y_type=row['y_type']
    )

print(f"图包含 {G.number_of_nodes()} 个节点和 {G.number_of_edges()} 条边")

In [ ]:
# 使用spring layout布局
pos = nx.spring_layout(G, k=1, iterations=50)

# 准备节点数据
node_x = []
node_y = []
node_text = []
node_color = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    # 确定节点类型
    node_type = 'unknown'
    for _, row in subset_df.iterrows():
        if row['x_id'] == node:
            node_type = row['x_type']
            break
        elif row['y_id'] == node:
            node_type = row['y_type']
            break
    
    node_text.append(f"{node}<br>类型: {node_type}<br>度: {G.degree(node)}")
    
    # 根据类型设置颜色
    if node_type == 'drug':
        node_color.append('red')
    elif node_type == 'disease':
        node_color.append('blue')
    else:
        node_color.append('gray')

# 准备边数据
edge_x = []
edge_y = []

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# 创建图形
fig = go.Figure()

# 添加边
fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
))

# 添加节点
fig.add_trace(go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        size=10,
        color=node_color,
        line=dict(width=2, color='white')
    )
))

fig.update_layout(
    title='药物-疾病关系网络 (Top 50 高度节点)',
    titlefont_size=16,
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20,l=5,r=5,t=40),
    annotations=[
        dict(
            text="红色: 药物, 蓝色: 疾病",
            showarrow=False,
            xref="paper", yref="paper",
            x=0.005, y=-0.002,
            xanchor="left", yanchor="bottom",
            font=dict(size=12)
        )
    ],
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

fig.show()

## 7. 数据质量分析

In [ ]:
# 检查缺失值
print("=== 缺失值检查 ===")
print(drug_disease_df.isnull().sum())

# 检查重复边
print("\n=== 重复边检查 ===")
duplicates = drug_disease_df.duplicated(subset=['x_id', 'y_id', 'display_relation'])
print(f"重复边数量: {duplicates.sum()}")

# 检查自环
print("\n=== 自环检查 ===")
self_loops = drug_disease_df[drug_disease_df['x_id'] == drug_disease_df['y_id']]
print(f"自环数量: {len(self_loops)}")

# 数据完整性
print("\n=== 数据完整性 ===")
print(f"唯一药物数量: {drug_disease_df[drug_disease_df['x_type'] == 'drug']['x_id'].nunique() + drug_disease_df[drug_disease_df['y_type'] == 'drug']['y_id'].nunique()}")
print(f"唯一疾病数量: {drug_disease_df[drug_disease_df['x_type'] == 'disease']['x_id'].nunique() + drug_disease_df[drug_disease_df['y_type'] == 'disease']['y_id'].nunique()}")

## 8. 总结

通过以上分析，我们可以得出以下结论：

1. **数据规模**: PrimeKG是一个大规模的生物医学知识图谱
2. **药物-疾病关系**: 包含多种类型的药物-疾病关系
3. **网络特性**: 具有典型的复杂网络特征，如度分布的长尾特性
4. **数据质量**: 数据质量较高，缺失值和重复值较少

这些特征使得PrimeKG非常适合用于药物-疾病关系预测任务。